<a href="https://colab.research.google.com/github/davidemichelon11/DL_Assignment/blob/main/DL_assignment_Giulio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries

In [ ]:
from google.colab import drive
import os
import shutil
from tqdm import tqdm
import torch
import torchvision
import torch.nn as nn
from torch.autograd import Function
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.tensorboard import SummaryWriter

# Extract data and create dataset

In [ ]:
drive.mount('/content/gdrive')

In [ ]:
!unzip -q -o gdrive/MyDrive/Adaptiope.zip 

In [ ]:
classes = ["/backpack", "/bookcase", "/car jack", "/comb", "/crown", "/file cabinet", "/flat iron", "/game controller", "/glasses",
           "/helicopter", "/ice skates", "/letter tray", "/monitor", "/mug", "/network switch", "/over-ear headphones", "/pen",
           "/purse", "/stand mixer", "/stroller"]

for d, td in zip(["Adaptiope/product_images", "Adaptiope/real_life"], ["adaptiope_small/product_images", "adaptiope_small/real_life"]):
  os.makedirs(td)
  for c in tqdm(classes):
    c_path = ''.join((d, c))
    c_target = ''.join((td, c))
    shutil.copytree(c_path, c_target)

# Create dataloader

In [ ]:
def get_data(batch_size, product_root, real_root):
  # resizing and cropping
  # prepare data transformations for the train loader
  transform = list()
  transform.append(T.Resize((256, 256)))                      # resize each PIL image to 256 x 256
  transform.append(T.RandomCrop((224, 224)))                 # randomly crop a 224 x 224 patch
  transform.append(T.ToTensor())                              # convert Numpy to Pytorch Tensor
  transform.append(T.Normalize(mean=[0.485, 0.456, 0.406], 
                               std=[0.229, 0.224, 0.225]))    # normalize with ImageNet mean
  transform = T.Compose(transform)                            # compose the above transformations into one
    
  # load data
  dataset_prod = torchvision.datasets.ImageFolder(root=product_root, transform=transform)
  dataset_real = torchvision.datasets.ImageFolder(root=real_root, transform=transform)
  
  # create train and test splits (80/20)
  num_samples = len(dataset_prod) # same number of samples in this dataset
  training_samples = int(num_samples * 0.8 + 1)
  test_samples = num_samples - training_samples

  train_data_prod, test_data_prod = torch.utils.data.random_split(dataset_prod, [training_samples, test_samples])
  train_data_real, test_data_real = torch.utils.data.random_split(dataset_real, [training_samples, test_samples])

  # initialize dataloaders
  train_loader_prod = torch.utils.data.DataLoader(train_data_prod, batch_size, shuffle=True)
  test_loader_prod = torch.utils.data.DataLoader(test_data_prod, batch_size, shuffle=False)
  
  train_loader_real = torch.utils.data.DataLoader(train_data_real, batch_size, shuffle=True)
  test_loader_real = torch.utils.data.DataLoader(test_data_real, batch_size, shuffle=False)
  
  return (train_loader_prod, test_loader_prod), (train_loader_real, test_loader_real)

# Create model

In [ ]:
class DLSA(torch.nn.Module):
  def __init__(self, num_classes):
    super().__init__()
    self.backbone = torchvision.models.resnet50(pretrained=True)

    self.linear1 = nn.Linear(1000, 512)
    self.batch_norm1 = nn.BatchNorm1d(512)

    self.linear2 = nn.Linear(512, 512)
    self.batch_norm2 = nn.BatchNorm1d(512)

    self.dropout = nn.Dropout()
    self.linear3 = nn.Linear(512, num_classes)
  
  def forward(self, x):
    x = self.backbone(x)
    
    x = F.relu(self.linear1(x))
    x = self.batch_norm1(x)

    x = F.relu(self.linear2(x))
    x = self.batch_norm2(x)

    x = self.dropout(x)
    g_x = self.linear3(x)

    y_x = F.softmax(g_x)
    
    return g_x, y_x

# Specify cost function and optimizer

In [ ]:
def get_cost_function():
  cost_function = torch.nn.CrossEntropyLoss()
  return cost_function

In [ ]:
def get_optimizer(model, lr, wd, momentum):
  
  # we will create two groups of weights, one for the newly initialized layer
  # and the other for rest of the layers of the network
  
  final_layer_weights = []
  rest_of_the_net_weights = []
  
  # iterate through the layers of the network
  for name, param in model.named_parameters():
    if name.startswith('fc'):
      final_layer_weights.append(param)
    else:
      rest_of_the_net_weights.append(param)
  
  # assign the distinct learning rates to each group of parameters
  optimizer = torch.optim.SGD([
      {'params': rest_of_the_net_weights},
      {'params': final_layer_weights, 'lr': lr}
  ], lr=lr/10, weight_decay=wd, momentum=momentum)
  
  return optimizer

# Train model

In [ ]:
def compute_a_z_b_z(g_z):
  L_v = g_z[..., 0:1] # 1st element of all samples
  L_w = g_z[..., 1:] # all elements, except 1st, of all samples
  L_v_mean = torch.mean(L_v, axis=0, keepdim=True) # mean over samples
  L_w_mean = torch.mean(L_w, axis=0, keepdim=True) # mean over samples

  nominator = torch.sum(L_v*L_w - L_v_mean*L_w_mean, axis=0, keepdim=True) / g_z.shape[0]
  denominator = torch.sum(L_v - L_v_mean**2, axis=0, keepdim=True) / g_z.shape[0]

  a_z = nominator / denominator
  b_z = L_w_mean - a_z*L_v_mean

  return a_z, b_z

In [ ]:
def compute_adaptation_loss(a_s, a_t, b_s, b_t, gamma):
  inner_product = (a_s * a_t).sum(dim=1)
  a_s_norm = torch.norm(a_s)
  a_t_norm = torch.norm(a_t)
  cos = inner_product / (a_s_norm * a_t_norm + 1e-9) # add small number to avoid dividing by 0
  angle = torch.acos(cos)

  return torch.deg2rad(angle) + gamma*torch.norm(b_s - b_t)**2

In [ ]:
def compute_conditional_loss(g_s, g_t, classes_s, classes_t, gamma):
  lc_loss = torch.zeros(1, device=device)
  num_classes = 0

  for c in torch.cat((classes_s, classes_t)).unique():
    g_s_c = g_s[(classes_s == c).nonzero(as_tuple=True)]
    g_t_c = g_t[(classes_t == c).nonzero(as_tuple=True)]

    # if one of the domains has less than one sample with class c, skip this class
    if not len(g_s_c) or not len(g_t_c):
      continue

    a_s_c, b_s_c = compute_a_z_b_z(g_s_c)
    a_t_c, b_t_c = compute_a_z_b_z(g_t_c)

    lc_loss += compute_adaptation_loss(a_s_c, a_t_c, b_s_c, b_t_c, gamma)
    num_classes += 1

  return lc_loss/num_classes if num_classes else lc_loss

In [ ]:
def training_step(model, source_train_loader, target_train_loader, optimizer, 
                  cost_function, device='cuda:0'):
  source_samples = 0.
  target_samples = 0.
  cumulative_ce_loss = 0.
  cumulative_accuracy = 0.

  gamma = 0.1
  alpha = 0.2
  
  target_iter = iter(target_train_loader)

  # strictly needed if network contains layers which has different behaviours between train and test
  model.train()
  pbar = tqdm(source_train_loader)

  mloss_ce = torch.zeros(1)
  mloss_lm = torch.zeros(1)
  mloss_lc = torch.zeros(1)
  for i, (inputs_source, targets) in enumerate(pbar):
    
    # get target data. If the target iterator reaches the end, restart it
    try:
      inputs_target, _ = next(target_iter)
    except:
      target_iter = iter(target_train_loader)
      inputs_target, _ = next(target_iter)
    
    inputs = torch.cat((inputs_source, inputs_target), dim=0)
    
    # load data into GPU
    inputs = inputs.to(device)
    targets = targets.to(device)
      
    # forward pass
    g_x, y_x = model(inputs)
    
    # split the source and target outputs
    g_s, g_t = torch.split(g_x, split_size_or_sections=inputs_source.shape[0], dim=0)
    y_s, y_t = torch.split(y_x, split_size_or_sections=inputs_source.shape[0], dim=0)
    
    a_s, b_s = compute_a_z_b_z(g_s)
    a_t, b_t = compute_a_z_b_z(g_t)

    # apply the losses
    ce_loss = cost_function(y_s, targets)
    lm_loss = compute_adaptation_loss(a_s, a_t, b_s, b_t, gamma)
    lc_loss = compute_conditional_loss(g_s, g_t, targets, torch.argmax(y_t, dim=1), gamma)
    
    loss = ce_loss + 0.01*lm_loss + 0.001*lc_loss
    
    # backward pass
    loss.backward()
    
    # update parameters
    optimizer.step()
    
    # reset the optimizer
    optimizer.zero_grad()

    # print statistics
    source_samples += inputs_source.shape[0]
    target_samples += inputs_target.shape[0]
    
    cumulative_ce_loss += ce_loss.item()
    _, predicted = y_s.max(1)
    cumulative_accuracy += predicted.eq(targets).sum().item()

    mloss_ce = (mloss_ce * i + ce_loss.item()) / (i + 1)
    mloss_lm = (mloss_lm * i + lm_loss.item()) / (i + 1)
    mloss_lc = (mloss_lc * i + lc_loss.item()) / (i + 1)

    pbar.set_description("CE loss {} | LM loss {} | LC loss {}".format(round(mloss_ce.item(),4), round(mloss_lm.item(),4), round(mloss_lc.item(),4)))

  return cumulative_ce_loss/source_samples, cumulative_accuracy/source_samples*100


def test_step(model, target_test_loader, cost_function, device='cuda:0'):
  samples = 0.
  cumulative_loss = 0.
  cumulative_accuracy = 0.

  # strictly needed if network contains layers which has different behaviours between train and test
  model.eval()

  with torch.no_grad():

    for batch_idx, (inputs, targets) in enumerate(tqdm(target_test_loader)):

      # load data into GPU
      inputs = inputs.to(device)
      targets = targets.to(device)
        
      # forward pass
      g_x, y_x = model(inputs)

      # apply the loss
      loss = cost_function(y_x, targets)

      # print statistics
      samples += inputs.shape[0]
      cumulative_loss += loss.item() # Note: the .item() is needed to extract scalars from tensors
      _, predicted = y_x.max(1)
      cumulative_accuracy += predicted.eq(targets).sum().item()

  return cumulative_loss/samples, cumulative_accuracy/samples*100

# Execute everything

In [ ]:
# Initialize random number generator (RNG) seeds https://pytorch.org/docs/stable/notes/randomness.html
# cudnn seed 0 settings are slower and more reproducible, else faster and less reproducible
seed = 0

import torch.backends.cudnn as cudnn
import random
import numpy as np
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
cudnn.benchmark, cudnn.deterministic = (False, True) if seed == 0 else (True, False)

In [ ]:
batch_size = 60
device='cuda:0'
learning_rate=0.01
weight_decay=0.000001
momentum=0.9
epochs=10

prod_root = 'adaptiope_small/product_images'
real_root = 'adaptiope_small/real_life'

dataloaders_prod, dataloaders_real = get_data(batch_size, prod_root, real_root)
train_loader_prod, test_loader_prod = dataloaders_real
train_loader_real, test_loader_real = dataloaders_prod

num_classes = len(set(train_loader_prod.dataset.dataset.targets))

model = DLSA(num_classes).to(device)

# model.load_state_dict(torch.load('./model2losses.pt'))

optimizer = get_optimizer(model, learning_rate, weight_decay, momentum)

cost_function = get_cost_function()

In [ ]:
for e in range(epochs):
  print('Epoch: {}/{}'.format(e+1, epochs))
  train_ce_loss, train_accuracy = training_step(model=model,
                                                source_train_loader=train_loader_prod,
                                                target_train_loader=train_loader_real,
                                                optimizer=optimizer, 
                                                cost_function=cost_function,
                                                device=device)
  
  test_loss, test_accuracy = test_step(model=model, 
                                       target_test_loader=test_loader_real, 
                                       cost_function=cost_function, 
                                       device=device)
  
  # print('Train: CE loss {:.5f}, Accuracy {:.2f}'.format(train_ce_loss, train_accuracy))
  print('Test: CE loss {:.5f}, Accuracy {:.2f}'.format(test_loss, test_accuracy))
  print('-----------------------------------------------------')

In [ ]:
# torch.save(model.state_dict(), './.pt')

# Debug